In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LassoCV

import statsmodels.api as sm
import statsmodels.formula.api as smf

import time

from Ames_Functions import *

pd.set_option("display.max_columns", None)

In [3]:
#Loading datasets
df_house2021 = load_state_pkl()["df_RE_latlong2021"][0].set_index("SaleID")
df_house2019 = load_state_pkl()["df_RE_latlong2019"][0].set_index("SaleID")
df_biz = pd.read_csv("data/df_EssentialLatLong_v2.csv")

#Imported from Ames_run_APIs notebook
df_driv_time_final2019 = load_state_pkl()["2019dfs_for_feature_engineering"][0].filter(regex = "_time")
df_driv_time_final2021 = load_state_pkl()["2021dfs_for_feature_engineering"][0].filter(regex = "_time")

# Finding differences in two different variables for feature engineering
## (SKIP)

In [439]:
# #Possible differences to consider:
# #High_School/Library
# #Sports_Venue/Restaurant
# #Movies/Restaurant
# #Park/Recreation
# #Organic_Groceries/Shopping
# #ISU/Library



# (df_driv_final["from_time"]
#     .eval("HS_Lib_diff = abs(High_School - Library)")
#     .eval("SV_Rest_diff = abs(Sports_Venue - Restaurant)")
#     .eval("Mov_Rest_diff = abs(Movies-Restaurant)")
#     .eval("Park_Rec_diff = abs(Park - Recreation)")
#     .eval("Groc_Shop_diff = abs(Organic_Groceries - Shopping)")
# ).filter(regex = "diff$").corr()

,HS_Lib_diff,SV_Rest_diff,Mov_Rest_diff,Park_Rec_diff,Groc_Shop_diff
HS_Lib_diff,1.000000,-0.727417,-0.547363,-0.152308,-0.219560
SV_Rest_diff,-0.727417,1.000000,0.848838,-0.122952,0.397418
Mov_Rest_diff,-0.547363,0.848838,1.000000,-0.274410,0.370700
Park_Rec_diff,-0.152308,-0.122952,-0.274410,1.000000,-0.468057
Groc_Shop_diff,-0.219560,0.397418,0.370700,-0.468057,1.000000


# Finding ratios of distances from two different services for feature engineering 
## (SKIP)

In [652]:
# #Possible ratios to consider:
# #High_School/Library
# #Sports_Venue/Restaurant
# #Movies/Restaurant
# #Park/Recreation
# #Organic_Groceries/Shopping
# #ISU/Library



# ratio_dict = {}

# for a, b in [("High_School", "Library"), ("Sports_Venue", "Restaurant"), ("Movies", "Restaurant"), ("Park", "Recreation"), ("Organic_Groceries", "Shopping"), ("ISU", "Library")]:
#       ratio_dict[f"{a}_{b}_ratio"] = (df_driv_final["from_time"]
#                                                 .apply(lambda x: x[a]/x[b] if x[a] > x[b] else x[b]/x[a], axis = 1)
#                                                 .rename(f"{a}_{b}_ratio")
#                                     )

# pd.DataFrame(ratio_dict).join(df_driv_final["from_time"].SalePrice)

,High_School_Library_ratio,Sports_Venue_Restaurant_ratio,Movies_Restaurant_ratio,Park_Recreation_ratio,Organic_Groceries_Shopping_ratio,ISU_Library_ratio,SalePrice
SaleID,,,,,,,
2,1.201318,2.804738,2.791816,1.018666,1.573722,1.159183,78000.0
3,1.201318,2.804738,2.791816,1.018666,1.573722,1.159183,408905.0
18,1.197453,2.743412,2.730929,1.018309,1.553475,1.156232,434066.0
19,1.196223,2.724280,2.711934,1.018196,1.547170,1.155292,88000.0
20,1.194971,2.704985,2.692777,1.018080,1.540816,1.154335,88000.0
...,...,...,...,...,...,...,...
22743,1.736798,1.551261,1.102166,1.041003,1.440054,1.712640,220000.0
22749,1.682007,1.504043,1.092605,1.038153,1.401589,1.659646,217500.0
22756,1.650223,1.477176,1.087238,1.036486,1.379769,1.628904,229000.0


### Evaluating the engineered features

In [ ]:
# EDA_report(pd.DataFrame(ratio_dict).drop(["Sports_Venue_Restaurant_ratio", "ISU_Library_ratio"], axis = 1).join(df_driv_final["from_time"].SalePrice))

In [665]:
# get_vif(pd.DataFrame(ratio_dict).drop(["Sports_Venue_Restaurant_ratio", "ISU_Library_ratio"], axis = 1))

[2.6474191730790873,
 2.0026501116710853,
 2.9995764720823432,
 1.1216954870182765]

# Lasso feature selection section using lassoCV

### 2021 section

In [5]:
#checking VIF for the drive time variables in df_driv_time_final
get_vif(df_driv_time_final2021.iloc[:, :-1])

[584.1548215648554,
 121.08663483485883,
 69.64328870942175,
 36.623979619912475,
 154.01923135898386,
 671.1610044528717,
 191.4066184464785,
 610.2068897850739,
 565.4533736344025,
 175.40341079311767,
 60.86777074535414,
 58.86435061209719,
 156.40891085897414,
 35.71323697339759,
 113.51633393284663,
 330.4767301731458,
 210.04214914956302]

In [13]:
#Defining X and y variables for lassoCV
X_lassoCV = {}
y_lassoCV = {}
X_lassoCV["from_time"] = df_driv_time_final2021
y_lassoCV["from_time"] = df_house2021.SalePrice

In [ ]:
#Fitting lassoCV model on X and y and checking which coefficients weren't reduced to 0
lassoCV = LassoCV(normalize = True)
lassoCV.fit(X_lassoCV["from_time"], y_lassoCV["from_time"])
lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]

In [16]:
#checking VIF for only the lassoCV selected features in df_drive_time_final
lassoCV_coefs = {}

lassoCV_coefs["from_time"] = df_driv_time_final2021[lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]]
lassoCV_coefs["from_time"].corr()

#get_vif(lassoCV_coefs["from_time"])

,Golf_time,Library_time,Movies_time,Recreation_time,Religion_time
Golf_time,1.000000,0.099523,0.107478,0.278473,0.516120
Library_time,0.099523,1.000000,0.814886,0.869053,0.543223
Movies_time,0.107478,0.814886,1.000000,0.699062,0.291242
Recreation_time,0.278473,0.869053,0.699062,1.000000,0.642763
Religion_time,0.516120,0.543223,0.291242,0.642763,1.000000


### 2019 section

In [13]:
#checking VIF for all df_driv_final variables
get_vif(df_driv_time_final2019.iloc[:, 2:-1])

[497.8720803675777,
 166.25262248853497,
 105.49663127559329,
 47.42152644218676,
 165.93510868566523,
 736.8679303864168,
 541.2549557538485,
 474.8168538915485,
 775.8228970286154,
 567.6047378056903,
 74.83131469946107,
 133.6042345276011,
 163.16991644494607,
 47.907261158854055,
 171.76357040297557,
 432.41773327566807,
 243.22226438952976,
 1123.4555166356288]

In [14]:
#Defining X and y variables for lassoCV
X_lassoCV = {}
y_lassoCV = {}
X_lassoCV["from_time"] = df_driv_time_final2019
y_lassoCV["from_time"] = df_house2019.SalePrice

In [ ]:
#Fitting lassoCV model on X and y and checking which coefficients weren't reduced to 0
lassoCV = LassoCV(normalize = True)
lassoCV.fit(X_lassoCV["from_time"], y_lassoCV["from_time"])
lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]

In [18]:
#checking VIF for only the lassoCV selected features in df_drive_time_final
lassoCV_coefs = {}

lassoCV_coefs["from_time"] = df_driv_time_final2019[lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]]
lassoCV_coefs["from_time"].corr()

#get_vif(lassoCV_coefs["from_time"])

,Golf_time,Library_time,Movies_time,Recreation_time,Religion_time
Golf_time,1.000000,0.233646,0.275352,0.433672,0.505963
Library_time,0.233646,1.000000,0.791193,0.841805,0.652546
Movies_time,0.275352,0.791193,1.000000,0.679968,0.396524
Recreation_time,0.433672,0.841805,0.679968,1.000000,0.747969
Religion_time,0.505963,0.652546,0.396524,0.747969,1.000000


# Find Homes that had all businesses nearby vs all homes that had businesses far away
## (SKIP)

In [ ]:
# #mean_shop = np.mean(df_driv_final["from_time"][["Shopping"]])[0]
# df_nearby = (df_driv_final["from_time"]
#     .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)[df_driv_final["from_time"].drop(["Lat_house", "Long_house", "SalePrice"], axis = 1).transform(lambda x: x < x.mean())]
#     .dropna()
#     #.isnull()
#     #.sum(axis = 0)
#     #.sort_values(ascending = False)
# ) #.plot.hist(alpha = 0.3) #.corr()

# df_far = (df_driv_final["from_time"]
#     .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)[df_driv_final["from_time"].drop(["Lat_house", "Long_house", "SalePrice"], axis = 1).transform(lambda x: x > x.mean())]
#     .dropna()
#     #.isnull()
#     #.sum(axis = 0)
#     #.sort_values(ascending = False)
# ) #.plot.hist(alpha = 0.3) #.corr()
# df_far.corr().Library #Library, Elem, 

In [ ]:
#get_vif(df_far[["Library", "Historic"]])

# Creating feature for average distance of closest 5 services to each house record

### 2021 Section

In [19]:
df_closest2021 = {}

for i in [1, 5, 10, 15]:
    df_closest2021[f"Closest_{i}"] = (df_driv_time_final2021
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .head(i)
        .sort_values(["SaleID", "driv_time"])
        .groupby("SaleID")
        .driv_time
        .mean()
    )

pd.DataFrame(df_closest2021)

,Closest_1,Closest_5,Closest_10,Closest_15
SaleID,,,,
2,85.5,210.28,324.33,403.220000
3,85.5,210.28,324.33,403.220000
18,95.3,220.08,334.13,413.020000
19,98.5,223.28,337.33,416.220000
20,101.8,226.58,340.63,419.520000
...,...,...,...,...
22743,276.6,295.28,333.13,363.680000
22749,305.2,323.88,361.73,392.280000
22756,324.0,342.68,380.53,411.080000


In [ ]:
#Checking EDA report for each closest 1,5,10,15 businesses to a home
EDA_closest2021 = {f"closest_{i}":EDA_report(pd.DataFrame(df_closest2021)[[f"Closest_{i}"]].join(df_house2021.SalePrice)) for i in [1,5,10,15]}

In [ ]:
#Checking EDA report results
EDA_closest2021['closest_5']

### 2019 Section

In [34]:
df_closest2019 = {}

for i in [1, 5, 10, 15]:
    df_closest2019[f"Closest_{i}"] = (df_driv_time_final2019
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .head(i)
        .sort_values(["SaleID", "driv_time"])
        .groupby("SaleID")
        .driv_time
        .mean()
    )

pd.DataFrame(df_closest2019).sort_values("Closest_1")

,Closest_1,Closest_5,Closest_10,Closest_15
SaleID,,,,
1419,6.3,130.28,187.84,228.200000
12929,7.8,339.40,404.03,450.320000
8324,8.4,55.84,80.44,136.620000
12927,8.4,339.52,404.03,450.200000
8342,12.8,57.86,79.48,136.573333
...,...,...,...,...
3906,371.0,455.70,501.02,547.180000
3867,371.1,455.80,501.12,547.280000
3920,378.0,462.70,508.02,554.180000


In [ ]:
#Checking EDA report for each closest 1,5,10,15 businesses to a home

EDA_closest2019 = {f"closest_{i}":EDA_report(pd.DataFrame(df_closest2019)[[f"Closest_{i}"]].join(df_house2019.SalePrice)) for i in [1,5,10,15]}

In [ ]:
#Checking EDA report results
EDA_closest2019['closest_5']

### Creating function to append desired results to final output for predictions notebook

In [39]:
def append_closest (df_input, df_times, s_closest_num):
    df = df_input.copy()
    df_temp = df_times.stack().reset_index().set_axis(["SaleID", "Service", "Closest_Time"], axis = 1).assign(Service = lambda x: x.Service.str.replace("_time", "_closest"))

    df = (df.join(pd.get_dummies(
                            df_temp.loc[df_temp.groupby("SaleID").Closest_Time.idxmin()],
                            columns = ["Service"]
                            )
                    .set_index("SaleID")
                )
            .join(s_closest_num)
        )
    return df

# Convenience Measurement Section (average proximity, measured in seconds, to all services)

### 2021 Section

In [43]:
s_convenience2021 = (df_driv_time_final2021
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .driv_time
        .mean()
        .sort_values()
        .rename("Convenience_secs")    
    )

pd.DataFrame(s_convenience2021)

,Convenience_secs
SaleID,
9379,134.555556
8302,137.072222
9371,139.594444
8227,143.994444
9382,144.144444
...,...
11354,618.622222
11336,620.538889
11334,621.338889


In [53]:
#Checking EDA report for convenience feature combined with closest_ features to measure their impact on Sale Price (i.e., R^2 and p-values)

EDA_report(pd.DataFrame(s_convenience2021).join(pd.DataFrame(df_closest2021)).join(df_house2021.SalePrice))

{'describe':        Convenience_secs   Closest_1   Closest_5  Closest_10  Closest_15  \
 count        633.000000  633.000000  633.000000  633.000000  633.000000   
 mean         356.660102  146.818799  216.309131  269.608167  318.602665   
 std          118.338210   84.844802   99.695220  111.015770  116.822033   
 min          134.555556    1.300000   20.880000   47.680000  102.893333   
 25%          250.172222   83.500000  136.700000  175.420000  213.113333   
 50%          360.616667  118.000000  201.840000  269.110000  318.920000   
 75%          447.522222  203.800000  297.360000  354.600000  405.180000   
 max          640.238889  371.800000  456.500000  533.470000  602.353333   
 
            SalePrice  
 count     633.000000  
 mean   258405.135861  
 std    115168.322605  
 min       900.000000  
 25%    178000.000000  
 50%    235000.000000  
 75%    317000.000000  
 max    793000.000000  ,
 'corr': SalePrice           1.000000
 Convenience_secs    0.345661
 Closest_15      

<Figure size 1008x1008 with 0 Axes>

In [48]:
#Checking VIFs for convenience feature against different Closest_ features from previous sections
get_vif(pd.DataFrame(s_convenience2021).join(pd.DataFrame(df_closest2021)))

[1807.228831608592,
 12.22192145109673,
 275.3490922700695,
 2240.7405283123653,
 5342.058876742899]

### 2019 Section

In [50]:
s_convenience2019 = (df_driv_time_final2019
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .driv_time
        .mean()
        .sort_values()
        .rename("Convenience_secs")
    )

pd.DataFrame(s_convenience2019)

,Convenience_secs
SaleID,
8229,144.972222
8230,145.150000
8427,147.944444
8260,152.594444
8291,153.816667
...,...
11035,638.338889
11266,638.938889
11268,639.438889


In [51]:
#Checking EDA report for convenience feature combined with closest_ features to measure their impact on Sale Price (i.e., R^2 and p-values)
EDA_report(pd.DataFrame(s_convenience2019).join(pd.DataFrame(df_closest2019)).join(df_house2019.SalePrice))

{'describe':        Convenience_secs   Closest_1   Closest_5  Closest_10  Closest_15  \
 count        901.000000  901.000000  901.000000  901.000000  901.000000   
 mean         370.669164  156.280244  230.535072  284.022575  332.961102   
 std          122.709903   91.405029  106.177163  116.475471  122.259921   
 min          144.972222    6.300000   36.920000   56.180000  110.820000   
 25%          268.266667   81.500000  137.080000  188.230000  230.566667   
 50%          373.433333  125.500000  213.460000  276.660000  330.193333   
 75%          462.655556  227.400000  326.120000  379.990000  431.000000   
 max          640.238889  392.600000  477.300000  533.470000  602.353333   
 
            SalePrice  
 count     901.000000  
 mean   226669.041065  
 std    109469.497483  
 min      4350.000000  
 25%    158855.000000  
 50%    213000.000000  
 75%    282240.000000  
 max    771530.000000  ,
 'corr': SalePrice           1.000000
 Convenience_secs    0.208342
 Closest_15      

<Figure size 1008x1008 with 0 Axes>

In [54]:
#Checking VIFs for convenience feature against different Closest_ features from previous sections
get_vif(pd.DataFrame(s_convenience2019).join(pd.DataFrame(df_closest2019)))

[1788.1257894507291,
 11.824684355523706,
 350.99258580033484,
 2827.9807540712964,
 5790.934643711074]

### Creating function to append desired results to final output for predictions notebook

In [68]:
def add_convenience_series (df_input, df_driv_time_final):
    df = df_input.copy()
    s_convenience = (df_driv_time_final
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .driv_time
        .mean()
        .sort_values()
        .rename("Convenience_secs")
        .apply(round, ndigits = 1)
    )
    return df.join(s_convenience)

# Centrality Measurement Section (Proximity to Old Town Historic District)

### Creating function to append desired result to final output for predictions notebook

In [69]:
def add_centrality_series (df_input, df_driv_time_final):
    df = df_input.copy()
    s_centrality = df_driv_time_final.Historic_time.rename("Central_historic_feature")

    return df.join(s_centrality)

## Count Number of Businesses within 1 minute of each house section

### 2021 Section

In [58]:
num_biz_dict2021 = {}

for i in np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])*60:
    num_biz_dict2021[f"biz_{i}_secs"] = (df_driv_time_final2021
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .groupby("SaleID")
        .driv_time
        .apply(lambda x: (x <= i).sum())
        .rename("count")
    )

pd.DataFrame(num_biz_dict2021)

,biz_60_secs,biz_120_secs,biz_180_secs,biz_240_secs,biz_300_secs,biz_360_secs,biz_420_secs,biz_480_secs,biz_540_secs,biz_600_secs
SaleID,,,,,,,,,,
2,0,1,2,2,5,6,6,8,12,13
3,0,1,2,2,5,6,6,8,12,13
18,0,1,2,2,5,6,6,8,12,13
19,0,1,2,2,4,6,6,8,12,13
20,0,1,2,2,4,6,6,8,12,13
...,...,...,...,...,...,...,...,...,...,...
22743,0,0,0,0,3,7,13,15,15,16
22749,0,0,0,0,0,5,10,14,15,16
22756,0,0,0,0,0,4,7,14,15,15


In [61]:
#Checking EDA report results for all num_biz features (mainly p-values)
print(EDA_report(pd.DataFrame(num_biz_dict2021).join(df_house2021.SalePrice))['ols_summary'])

#Found that 3mins and 8mins were only significant p-values (<0.05)

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     10.71
Date:                Thu, 14 Sep 2023   Prob (F-statistic):           7.12e-17
Time:                        14:41:09   Log-Likelihood:                -8224.5
No. Observations:                 633   AIC:                         1.647e+04
Df Residuals:                     622   BIC:                         1.652e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     4.476e+05   4.11e+04     10.877   

<Figure size 1008x1008 with 0 Axes>

In [63]:
#Checking VIF for 3mins and 8mins features
get_vif(pd.DataFrame(num_biz_dict)[["biz_180_secs", "biz_480_secs"]])

[2.6345201515612775, 2.6345201515612775]

### 2019 Section

In [65]:
num_biz_dict2019 = {}

for i in np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])*60:
    num_biz_dict2019[f"biz_{i}_secs"] = (df_driv_time_final2019
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .groupby("SaleID")
        .driv_time
        .apply(lambda x: (x <= i).sum())
        .rename("count")
    )

pd.DataFrame(num_biz_dict2019)

,biz_60_secs,biz_120_secs,biz_180_secs,biz_240_secs,biz_300_secs,biz_360_secs,biz_420_secs,biz_480_secs,biz_540_secs,biz_600_secs
SaleID,,,,,,,,,,
92,0,0,0,0,0,2,4,6,10,11
93,0,0,0,0,0,2,4,6,10,11
96,0,0,0,0,1,3,7,8,14,15
98,0,0,0,0,1,3,7,8,13,15
100,0,0,0,0,1,2,7,8,11,15
...,...,...,...,...,...,...,...,...,...,...
22745,0,0,0,0,2,5,12,14,15,16
22747,0,0,0,0,2,5,11,14,15,16
22750,0,0,0,0,0,5,9,14,15,16


In [66]:
#Checking EDA report results for all num_biz features (mainly p-values)
print(EDA_report(pd.DataFrame(num_biz_dict2019).join(df_house2019.SalePrice))['ols_summary'])

#Found (again) that 3mins and 8mins were only significant p-values (<0.05)

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     5.823
Date:                Thu, 14 Sep 2023   Prob (F-statistic):           1.55e-08
Time:                        14:44:45   Log-Likelihood:                -11704.
No. Observations:                 901   AIC:                         2.343e+04
Df Residuals:                     890   BIC:                         2.348e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     3.199e+05   2.76e+04     11.608   

<Figure size 1008x1008 with 0 Axes>

In [67]:
#Checking VIF for 3mins and 8mins features
get_vif(pd.DataFrame(num_biz_dict2019)[["biz_180_secs", "biz_480_secs"]])

[2.493339804500264, 2.493339804500263]

### Creating function to append desired results to final ouput for predictions notebook

In [70]:
def add_num_biz_by_time (df_input, df_driv_time_final, times):
        df = df_input.copy()
        for t in times: 
                num_biz = (df_driv_time_final
                #.drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
                .stack()
                .reset_index()
                .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
                .groupby("SaleID")
                .driv_time
                .apply(lambda x: (x <= t).sum())
                .rename(f"biz_{t}_secs")
                )
                df = df.join(num_biz)
        return df

## Count the number of houses around each service

## (SKIP)

In [ ]:
# num_house_dict = {}

# for s in np.array([2, 3, 4])*60:
#     num_house_dict[f"house_{s}_secs"] = (df_driv_final["from_time"]
#         .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
#         .stack()
#         .reset_index()
#         .set_axis(["SaleID", "Service", "driv_time_secs"], axis = 1)
#         .groupby('Service')
#         .apply(lambda x: x[x.driv_time_secs < s].driv_time_secs.count())
#     )

# pd.DataFrame(num_house_dict).join(df_biz.Service.value_counts().rename("Service_count")).apply(lambda x: round(x/x.Service_count, 2), axis = 1).drop("Service_count", axis = 1)

In [ ]:
# mean_SalePrice = {}

# for s in np.array([2, 3, 4])*60:
#     mean_SalePrice[f"mean_salePrice_{s}_secs"] = (df_driv_final["from_time"]
#         .drop(["Lat_house", "Long_house"], axis = 1)
#         .set_index("SalePrice")
#         .stack()
#         .reset_index()
#         .set_axis(["SalePrice", "Service", "driv_time_secs"], axis = 1)
#         .groupby('Service')
#         .apply(lambda x: x[x.driv_time_secs < s].SalePrice.mean())
#         .transform(round, ndigits = 2)
#     )

# pd.DataFrame(mean_SalePrice).sort_values(by = "mean_salePrice_120_secs")

In [832]:
# df_nearby_houses = (pd.DataFrame(mean_SalePrice)
#     .join(pd.DataFrame(num_house_dict)
#           .join(df_biz.Service.value_counts().rename("Service_count"))
#           .apply(lambda x: round(x/x.Service_count, 2), axis = 1)
#           .drop("Service_count", axis = 1))
#     .sort_values('mean_salePrice_120_secs')
# )

In [ ]:
# get_vif(df_nearby_houses.dropna()[["mean_salePrice_120_secs", "house_120_secs"]])

## Housing Amenities Feature Engineering

In [71]:
def transform_RE_features (df_input):
    df_RE = df_input.copy()

    # Adding Fields "Has_Fireplace" and "Has_Garage" and "Has_Bsmt"
    has_feature = lambda x: 1 if x > 0 else 0
    df_RE["Has_Fireplace"] = df_RE.Fireplaces.apply(has_feature)
    df_RE["Has_Garage"] = df_RE.GarageArea.apply(has_feature)
    df_RE["Has_Bsmt"] = df_RE.TtlBsmtSF.apply(has_feature)

    # Adding fields: TotalArea and TotalArea_Gar
    df_RE = df_RE.eval("TotalArea = GLA + TtlBsmtSF	").eval("TotalArea_Gar = GLA + TtlBsmtSF	+ GarageArea")
    
    # Adding Field TotalExt and dummy variables for it
    df_Ext_dummies = pd.get_dummies(df_RE.Ext2).rename(columns = {"Brk Cmn": "BrkComm", "CmentBd": "CemntBd", "Wd Shng": "WdShing"})

    for col in df_RE.Ext1.unique():
        df_Ext_dummies[[col]] += pd.get_dummies(df_RE.Ext1)[[col]]

    df_Ext_dummies.rename(columns = {"C'Block": "CBlock"}, inplace = True)
    df_Ext_dummies.rename(columns = {col:col + "_Ext" for col in df_Ext_dummies.columns}, inplace = True)
    df_RE = df_RE.join(df_Ext_dummies)

    # Adding dummy variable columns for GarageType
    clean_colname = lambda x: x.replace("-", "_").replace(".", "_").replace("/", "_").replace(" ", "_")
    df_RE = df_RE.join(pd.get_dummies(df_RE.GarageType).rename(columns = {col:clean_colname(col) + "_GrType" for col in pd.get_dummies(df_RE.GarageType).columns}))

    # Adding dummy variable columns for HouseStyle and renaming leading numbers with words
    df_RE = df_RE.join(pd.get_dummies(df_RE.HouseStyle).rename(columns = {col:clean_colname(col) + "_HsStyle" for col in pd.get_dummies(df_RE.HouseStyle).columns}))
    df_RE.rename(columns = {col: col.replace(" ", "_").replace("1", "One").replace("2", "Two").replace("5", "Five") for col in df_RE.columns}, inplace = True)

    return df_RE

# Creating Final DataFrames including all Engineering Features for Output to Predictions Notebook

In [72]:
#Preparing df_feature_all for entry into pickle file for 2019, 2021 and both combined into one master dataset (df_feature_all)
df_feature_all2019 = (df_house2019
    .pipe(transform_RE_features)
    .filter(regex = "Yr|_Ext|Has|Area|SF|Cars|Fireplaces|GrType|HsStyle").drop(["YrSold_YYYY"], axis = 1)
    .pipe(append_closest, df_driv_time_final2019, pd.DataFrame(df_closest2019).Closest_5)
    .pipe(add_convenience_series, df_driv_time_final2019)
    .pipe(add_centrality_series, df_driv_time_final2019)
    .pipe(add_num_biz_by_time, df_driv_time_final2019, [180, 480])
)

df_feature_all2021 = (df_house2021
    .pipe(transform_RE_features)
    .filter(regex = "Yr|_Ext|Has|Area|SF|Cars|Fireplaces|GrType|HsStyle").drop(["YrSold_YYYY"], axis = 1)
    .pipe(append_closest, df_driv_time_final2021, pd.DataFrame(df_closest2021).Closest_5)
    .pipe(add_convenience_series, df_driv_time_final2021)
    .pipe(add_centrality_series, df_driv_time_final2021)
    .pipe(add_num_biz_by_time, df_driv_time_final2021, [180, 480])
)

df_feature_all_both = df_feature_all2019.eval("Year = 2019").append(df_feature_all2021.eval("Year = 2021")).fillna(0)

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_2331/3605012601.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_feature_all_both = df_feature_all2019.eval("Year = 2019").append(df_feature_all2021.eval("Year = 2021")).fillna(0)


### Saving to Ames_notebook_state pkl file

In [321]:
desc = "post feature engineering including boolean fields for Fireplace/Gar/Bsmt, dummy fields for Ext, GrType and HouseStyle, TotalArea/TotalArea_Gar fields, and drive time related fields (Closest_5, Convenience etc)"

save_state_pkl("df_house2021_after_feature_engineering",
                df_feature_all2021,
                "df_house2021 " + desc
                )

save_state_pkl("df_house2019_after_feature_engineering",
                df_feature_all2019,
                "df_house2019 " + desc
                )

save_state_pkl("df_house_both_after_feature_engineering",
                df_feature_all_both,
                "df_house2019 + df_house2021 " + desc
                )